In [160]:
import json
import datetime

from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, SystemMessage # AIMessage, ToolMessage (not explicitly used in this simplified version)
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate

from datetime import datetime
# from langchain_core.pydantic_v1 import BaseModel, Field # Simplified parsing won't use Pydantic tools for this version

In [161]:
# from src.config import Config
from src.mock_apis import room_services, booking_services
from src.schemas import AgentState, BookingRequest
from src.helper import get_llm, validate_request

In [162]:
from IPython.display import display, JSON, Markdown, Image

# class BookingRequest(BaseModel):
#     start_date: Optional[str] = None  # Format: ISO
#     duration_hours: Optional[int] = None
#     capacity: Optional[int] = None
#     equipments: Optional[List[str]] = None
#     user_name: Optional[str] = None
#     clarification_needed: bool
#     clarification_question: Optional[str] = None

parser = PydanticOutputParser(pydantic_object=BookingRequest)
parsing_schema = parser.get_format_instructions()

Markdown(parsing_schema)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"start_time": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "description": "ISO format, e.g., 2025-07-16T10:00:00", "title": "Start Time"}, "duration_hours": {"anyOf": [{"type": "integer"}, {"type": "null"}], "default": null, "description": "Duration in hours", "title": "Duration Hours"}, "capacity": {"anyOf": [{"type": "integer"}, {"type": "null"}], "default": null, "description": "Room capacity", "title": "Capacity"}, "equipments": {"anyOf": [{"items": {"type": "string"}, "type": "array"}, {"type": "null"}], "default": null, "description": "List of equipment", "title": "Equipments"}, "user_name": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "description": "Name of the user", "title": "User Name"}, "clarification_needed": {"default": false, "description": "Whether clarification is needed", "title": "Clarification Needed", "type": "boolean"}, "clarification_question": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "description": "Question for clarification", "title": "Clarification Question"}}}
```

In [163]:
# Get tomorrow's date
current_date = (datetime.now()).strftime('%Y-%m-%d')

TEMPLATE = "\n".join([
        "You are a meeting room assistant. Extract the following fields from the user request.",
        
        "#### Required fields:",
            "- start_time: If the user not specifies the date, (e.g., \"tomorrow\" or something similar), calculate the time from {current_date}.",
            "- duration_hours: (e.g., 2)\n",
            "- capacity: integer (e.g., \"for 5 people\" → 5)",
            "- equipments: list of strings (e.g., [\"projector\", \"whiteboard\"])",
            "- user_name: name of the person booking.\n",

        "### Instructions:",
            "1. Respond ONLY with a JSON object matching this schema: {parsing_schema}",
            "2. You **MUST respond ONLY with this JSON directly**.",
            "3. Do NOT include any explanatory text before or after the JSON.",
            "4. If fields are missing, set them to null.",
            "5. If start_time, capacity, equipments, user_name, or duration_hours is missing/unclear:",
            "   - Set \"clarification_needed\": true and ",
            "   - Set \"clarification_question\", ask for clarification about the missing fields\n",

        "### Example Outputs:",
            "- Complete info:\n{successful_example}",
            "- Needs clarification:\n{missing_example}\n",
        
        "Now process this request:\n\"{user_request}\""
        
    ])

In [164]:
# Example Python dicts
successful_example = {
    "start_time": "2025-07-16T10:00:00",
    "duration_hours": 1,
    "capacity": 3,
    "equipments": ["whiteboard"],
    "user_name": "Omar",
    "clarification_needed": False,
    "clarification_question": None
}

missing_example = {
    "clarification_needed": True,
    "clarification_question": "Could you tell me the meeting's start time and duration?"
}

In [165]:
prompt_template = PromptTemplate(
    input_variables=["user_request", "current_date"],
    template=TEMPLATE,
    partial_variables= {
    "parsing_schema": parsing_schema,
    "successful_example": json.dumps(successful_example, indent=2),
    "missing_example": json.dumps(missing_example, indent=2)
}
)

In [166]:
def parse_request(state: AgentState) -> AgentState:
    print("---NODE: PARSE REQUEST---")
    # initialize parser
    parser = PydanticOutputParser(pydantic_object=BookingRequest)

    current_date = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    llm = get_llm(name="groq")
    # prompt = prompt_template.format(user_request=state.get("original_request"), 
    #                                 current_date=current_date)
    # response = llm.invoke(prompt).content
    # parsed_data = parser.parse(response)
    chain = prompt_template | llm | parser
    parsed_data = chain.invoke({"user_request": state["original_request"],
                                    "current_date": current_date})
    state.update({
            "parsed_request": parsed_data.model_dump(),
            "clarification_needed": parsed_data.clarification_needed,
            "clarification_question": parsed_data.clarification_question,
            "user_name_for_booking": parsed_data.user_name,
            "error_message": None
        })
        
    return state

In [167]:
# Check if request is valid and clear
def is_clear_request(state: AgentState) -> str:
    print("---CONDITION: Check Valid and Clear Request---")
    if state.get("error_message"):
        print("--> Error detected")
        return "handle_error"
    elif state.get("clarification_needed"):
        print("--> Clarification needed")
        return "ask_clarification"
    else:
        print("--> Ready to book room")
        return "book_room"

In [168]:
def ask_clarification(state: AgentState) -> AgentState:
    print("---NODE: ASK CLARIFICATION---")
    state["final_response_to_user"] = state["clarification_question"]
    return state

In [169]:
def book_room(state: AgentState) -> AgentState:
    # In a real workflow, you'd call your booking API here
    state["final_response_to_user"] = "Booking room..."
    return state

In [170]:
def handle_error(state: AgentState) -> AgentState:
    state["final_response_to_user"] = state.get("error_message", "An unknown error occurred.")
    return state

In [171]:
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("parse_request_node", parse_request)
workflow.add_node("ask_clarification_node", ask_clarification)
workflow.add_node("book_room_node", book_room)
workflow.add_node("handle_error_node", handle_error)

# Set entry point
workflow.set_entry_point("parse_request_node")

# Add edges
workflow.add_conditional_edges(
    "parse_request_node",
    is_clear_request,
    {
        "ask_clarification": "ask_clarification_node",
        "book_room": "book_room_node",
        "handle_error": "handle_error_node"
    }
)

workflow.add_edge("ask_clarification_node", "parse_request_node")  # Loop back after clarification
workflow.add_edge("book_room_node", END)
workflow.add_edge("handle_error_node", END)

# Compile the graph
app = workflow.compile()

In [172]:
result = app.invoke({
    "original_request": 
    "Book a room for 5 people with a projector tomorrow at 3 PM for 2 hours"
})

---NODE: PARSE REQUEST---


---CONDITION: Check Valid and Clear Request---
--> Clarification needed
---NODE: ASK CLARIFICATION---
---NODE: PARSE REQUEST---
---CONDITION: Check Valid and Clear Request---
--> Clarification needed
---NODE: ASK CLARIFICATION---
---NODE: PARSE REQUEST---
---CONDITION: Check Valid and Clear Request---
--> Clarification needed
---NODE: ASK CLARIFICATION---
---NODE: PARSE REQUEST---
---CONDITION: Check Valid and Clear Request---
--> Clarification needed
---NODE: ASK CLARIFICATION---
---NODE: PARSE REQUEST---
---CONDITION: Check Valid and Clear Request---
--> Clarification needed
---NODE: ASK CLARIFICATION---
---NODE: PARSE REQUEST---
---CONDITION: Check Valid and Clear Request---
--> Clarification needed
---NODE: ASK CLARIFICATION---
---NODE: PARSE REQUEST---
---CONDITION: Check Valid and Clear Request---
--> Clarification needed
---NODE: ASK CLARIFICATION---
---NODE: PARSE REQUEST---
---CONDITION: Check Valid and Clear Request---
--> Clarification needed
---NODE: ASK CLARIFICATION---
--

KeyboardInterrupt: 

In [174]:
Markdown(result["final_response_to_user"])

Parsing failed: Invalid json output: I'll extract the required fields and process the request.

**Required fields:**

* `start_time`: "tomorrow" (assuming it means the next day, which is 2025-05-12)
* `duration_hours`: 2
* `capacity`: "for 5 people" (integer value: 5)
* `equipments`: ["projector"]
* `user_name`: not specified (assuming it's an empty string or null)

**Output:**

{
  "start_time": "2025-05-12T15:00:00",
  "duration_hours": 2,
  "capacity": 5,
  "equipments": [
    "projector"
  ],
  "user_name": "",
  "clarification_needed": false,
  "clarification_question": null
}

Note: I assumed the "tomorrow" means the next day, which is 2025-05-12. If the user meant something else, please clarify.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 